1. Load the packages

In [ ]:
import scipy.io
import random
import numpy as np
import torch
import os
import torchvision
import time
from torchvision import models,datasets, transforms
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.datasets import fetch_openml
import torch.nn as nn
from torch.autograd.function import Function
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler


2. Load Office-Caltech-10 dataset


You need to download [Office-Caltech-10 DeCAF-fc6 features](https://drive.google.com/drive/folders/1XAKj5ikc6ygcanexeiWuwvnHMf5rUVGy) generated in [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put the data in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put the data in the corresponding folder of your device.



[1] Donahue, J., Jia, Y., Vinyals, O., Hoffman, J., Zhang, N., Tzeng, E., & Darrell, T. (2014, January). Decaf: A deep convolutional activation feature for generic visual recognition. In International conference on machine learning (pp. 647-655). PMLR.

In [ ]:
# Mount the Google drive, please ignore this cell if you run the code locally.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
amazon = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/amazon_fc6.mat')
webcam = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/webcam_fc6.mat')
dslr = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/dslr_fc6.mat')
cal  = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/caltech_decaf.mat')


In [ ]:
amazon_fts,amazon_labels=amazon['fts'],amazon['labels'].reshape(-1)-1
webcam_fts,webcam_labels=webcam['fts'],webcam['labels'].reshape(-1)-1
dslr_fts,dslr_labels=dslr['fts'],dslr['labels'].reshape(-1)-1
cal_fts,cal_labels=cal['feas'],cal['labels'].reshape(-1)-1

In [ ]:
print(cal_labels)
print(webcam_labels.shape)
print(dslr_labels.shape)
print(cal_labels.shape)
n_class=10

[0 0 0 ... 9 9 9]
(795,)
(498,)
(1123,)


3. Define functions to formulate the training and testing sets

In [ ]:
np.random.choice
class_set=np.array([1,2,6,11,12,13,16,17,18,23])-1
#backpack, bike, calculator, headphones, keyboard, laptop-computer, monitor, mouse, mu, and projector

def get_data(domain_name):
  index=np.array([])
  label_for_training=[]
  fts,labels=None, None
  if domain_name=='amazon':
    fts,labels=amazon_fts,amazon_labels
  elif domain_name=='webcam':
    fts,labels=webcam_fts,webcam_labels
  elif domain_name=='dslr':
    fts,labels=dslr_fts,dslr_labels

  j=0
  for i in class_set:
    index=np.hstack((index,np.where(labels==i)[0]))
    for a in np.where(labels==i)[0]:
      label_for_training.append(j)
    j=j+1
  index=index.astype(int)
  return fts[index],np.array(label_for_training)

def get_index(feature,labels,select_num):
  select_list=np.array([])
  remaining_list=np.array([])
  for i in range(10):
    full_list=np.where(labels==i)[0]
    ran_select=np.random.choice(full_list,select_num,replace=False)
    select_list=np.hstack((select_list,ran_select))

    remain_list=np.setdiff1d(full_list,ran_select)
    remaining_list=np.hstack((remaining_list,remain_list))
  return select_list.astype(int),remaining_list.astype(int)


def souce_target_split(src_name, tar_name):
  if src_name =='cad':
    src_fea,src_label=np.array(cal_fts),np.array(cal_labels)
  else:
    src_fea,src_label=get_data(src_name)
  if tar_name == 'cad':
    tar_fea,tar_label=np.array(cal_fts),np.array(cal_labels)
  else:

    tar_fea,tar_label=get_data(tar_name)
  if src_name=='amazon':
    scr_number=20
  elif src_name=='cad':
    scr_number=20
  else:
    scr_number=8
  tar_number=3
  
  src_select,src_remain=get_index(src_fea,src_label,scr_number)
  tar_select,tar_remain=get_index(tar_fea,tar_label,tar_number)

  return src_fea[src_select],src_label[src_select],tar_fea[tar_select],tar_label[tar_select],tar_fea[tar_remain],tar_label[tar_remain]



4. Self-defined layer for implementing Center Loss

In [ ]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(CenterLoss, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label, centers, batch_size):
        ctx.save_for_backward(feature, label, centers, batch_size)
        centers_batch = centers.index_select(0, label.long())
        return (feature - centers_batch).pow(2).sum() / 2.0 / batch_size

    @staticmethod
    def backward(ctx, grad_output):
        feature, label, centers, batch_size = ctx.saved_tensors
        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())

        counts = counts.scatter_add_(0, label.long(), ones)
        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)
        return - grad_output * diff / batch_size, None, grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = CenterLoss(10,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1]).to(device)
    feat = torch.zeros(4,2).to(device).requires_grad_()
    print (list(ct.parameters()))
    
    print (ct.centers.grad)
    out = ct(y,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    main(test_cuda=False)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.1878, -0.9564],
        [-0.2440, -0.4153],
        [ 0.3259, -1.6059],
        [-0.5272,  0.3401],
        [-1.6526,  0.2108],
        [-1.3302,  0.4676]], requires_grad=True)]
None
0.8543729186058044
--------------------------------------------------------------------------------
[Parameter containing:
tensor([[ 0.9147, -1.1896],
        [-0.7501, -1.5465],
        [-0.4303,  0.3703],
        [ 0.4588,  0.2950],
        [ 0.2126, -0.1098],
        [-0.1349,  2.0038],
        [ 1.8339,  2.3544],
        [ 0.3990, -0.3584],
        [-0.2116,  0.5276],
        [ 0.5683, -0.6786]], device='cuda:0', requires_grad=True)]
None
0.9725638628005981


5. Define the network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(4096, 1024)
        self.act1= nn.PReLU()
        self.layer2 = nn.Linear(1024, 128)
        self.act2= nn.PReLU()
        self.ip2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 4096)
        x = self.layer1(x)
        x = self.act1(x)
        feature = self.layer2(x)
        feature = self.act2(feature)
        ip2 = self.ip2(feature)
        return feature , ip2

    def predict(self, x):
        return self.forward(x)

6. Define the function for training and testing 

In [ ]:
def trainandtest(model, dataloaders, optimizer,loss_weight):
    device = torch.device("cuda")
    centerloss = CenterLoss(31, 128).to(device)
    optimzer4center = torch.optim.SGD(centerloss.parameters(), lr =0.5)
    n_epoch = 150
    criterion = nn.CrossEntropyLoss()
    early_stop = 20
    since = time.time()
    best_acc = 0
    stop = 0
    for epoch in range(0, n_epoch):
        stop += 1
        for phase in ['src']:
            if phase == 'src':
                model.train()
            else:
                model.eval()
            total_loss, total_center,correct = 0, 0,0
            for inputs, labels in dataloaders[phase]:
                inputs, labels= inputs.cuda(),labels.cuda()
                optimizer.zero_grad()
                optimzer4center.zero_grad()


    
                with torch.set_grad_enabled(phase == 'src'):
                    ip1, outputs = model(inputs)
                    softmax=criterion(outputs, labels)
                    centerlss=centerloss(labels,ip1)
                    loss = softmax + loss_weight * centerlss
                preds = torch.max(outputs, 1)[1]
                if phase == 'src':
                    loss.backward()
                    optimizer.step()
                    optimzer4center.step()
                total_loss += softmax.item() * inputs.size(0)
                total_center +=centerlss.item()
                correct += torch.sum(preds == labels.data)
            epoch_loss = total_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct.double() / len(dataloaders[phase].dataset)

        model.eval()
        correct = 0
        for inputs, labels in dataloaders['tar']:
          inputs, labels= inputs.cuda(),labels.cuda()
          ip1, outputs = model(inputs)
          preds = torch.max(outputs, 1)[1]
          correct += torch.sum(preds == labels.data)
        epoch_acc = correct.double() / len(dataloaders['tar'].dataset)
        if epoch>145:
          print('Testing acc:',epoch_acc.item())
          print(f'Epoch: [{epoch:02d}/{n_epoch:02d}]---{phase}, softmax_loss: {epoch_loss:.6f}')
          print_center = total_center / len(dataloaders[phase].dataset)
          print('centerloss:',print_center)

    time_pass = time.time() - since
    print(f'Training complete in {time_pass // 60:.0f}m {time_pass % 60:.0f}s')


7. Define evaluation function

In [ ]:
def evaluate(types,domain_src,domain_tar,weight,seeds):
  batch_size = 32
  n_class=10
  random.seed(seeds)
  train_x_scr,train_y_scr,train_x_tar,train_y_tar,test_x_tar,test_y_tar=souce_target_split(domain_src, domain_tar)
  if types=="sourceonly":
    x_train=train_x_scr
    y_train=train_y_scr
  else:
    x_train=np.vstack((train_x_scr,train_x_tar)) 
    y_train=np.hstack((train_y_scr,train_y_tar))
  print(len(x_train))
  
  x_test=test_x_tar
  y_test=test_y_tar
  print(len(x_test))
  x_train=torch.from_numpy(x_train)
  y_train=torch.from_numpy(y_train)
  x_test=torch.from_numpy(x_test)
  y_test=torch.from_numpy(y_test)
  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()
  # form the dataset
  train_dataset=TensorDataset(x_train,y_train)
  val_dataset=TensorDataset(x_test,y_test)
  train_loader= DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=2)
  test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
  model = Net().cuda()
  param_group = []
  learning_rate = 0.0001
  momentum = 5e-4
  for k, v in model.named_parameters():


    if k.__contains__('classifier'):
        param_group += [{'params': v, 'lr': learning_rate}]
    elif k.__contains__('ip'):
        param_group += [{'params': v, 'lr': learning_rate}]

    else:
        param_group += [{'params': v, 'lr': learning_rate}]


  print('Types:',types)
  # optimizer = torch.optim.SGD(param_group, momentum=momentum)
  optimizer = torch.optim.Adam(param_group)
  dataloaders = {'src': train_loader,
               'tar': test_loader}
  trainandtest(model, dataloaders, optimizer,weight)


8. Example

Examples below are baseline Models 1, 2, and 3 on the *Amaon and DSLR* domain adaptation task.
You may change the value of variables to get other experimental results.

Note that models will be trained from scratch. Training time should last for around **6 minutes** for one repetition if you use the GPU (GeForce RTX 3090).

Although the results you get may be slightly different from the ones of the manuscript due to randomized initialization, the gap should be small.

In [ ]:
#'amazon', 'webcam','dslr','cad'
types='sourceonly'
source_domain='amazon'
target_domain='dslr'
random_seed=1
weight=0
evaluate(types,source_domain,target_domain,weight,random_seed)

200
127
Types: sourceonly
Testing acc: 0.6929133858267716
Epoch: [146/150]---src, softmax_loss: 0.000050
centerloss: 16.87419448852539
Testing acc: 0.6929133858267716
Epoch: [147/150]---src, softmax_loss: 0.000049
centerloss: 17.22310592651367
Testing acc: 0.6929133858267716
Epoch: [148/150]---src, softmax_loss: 0.000049
centerloss: 18.52044708251953
Testing acc: 0.6929133858267716
Epoch: [149/150]---src, softmax_loss: 0.000048
centerloss: 17.246609344482422
Training complete in 0m 48s


In [ ]:
#'amazon', 'webcam','dslr','cad'
types='softmax'
source_domain='amazon'
target_domain='dslr'
random_seed=1
weight=0
evaluate(types,source_domain,target_domain,weight,random_seed)


230
127
Types: softmax
Testing acc: 0.8740157480314961
Epoch: [146/150]---src, softmax_loss: 0.000092
centerloss: 14.088780411430028
Testing acc: 0.8740157480314961
Epoch: [147/150]---src, softmax_loss: 0.000091
centerloss: 14.437496682871943
Testing acc: 0.8818897637795275
Epoch: [148/150]---src, softmax_loss: 0.000090
centerloss: 14.078015932829484
Testing acc: 0.8818897637795275
Epoch: [149/150]---src, softmax_loss: 0.000089
centerloss: 14.476233706266983
Training complete in 0m 60s


In [ ]:
#'amazon', 'webcam','dslr','cad'
types='center loss'
source_domain='amazon'
target_domain='dslr'
random_seed=1
weight=0.01
evaluate(types,source_domain,target_domain,weight,random_seed)

230
127
Types: center loss
Testing acc: 0.9291338582677166
Epoch: [146/150]---src, softmax_loss: 0.003837
centerloss: 0.054520971878715185
Testing acc: 0.9212598425196851
Epoch: [147/150]---src, softmax_loss: 0.003775
centerloss: 0.05364571706108425
Testing acc: 0.9212598425196851
Epoch: [148/150]---src, softmax_loss: 0.003773
centerloss: 0.058090983266415805
Testing acc: 0.9133858267716535
Epoch: [149/150]---src, softmax_loss: 0.003735
centerloss: 0.05936995952025704
Training complete in 1m 4s


In [ ]:
#'amazon', 'webcam','dslr','cad'
types='sourceonly'
source_domain='dslr'
target_domain='amazon'
random_seed=1
weight=0
evaluate(types,source_domain,target_domain,weight,random_seed)

80
928
Types: sourceonly
Testing acc: 0.7079741379310345
Epoch: [146/150]---src, softmax_loss: 0.000045
centerloss: 11.218095779418945
Testing acc: 0.7079741379310345
Epoch: [147/150]---src, softmax_loss: 0.000045
centerloss: 11.00118408203125
Testing acc: 0.7079741379310345
Epoch: [148/150]---src, softmax_loss: 0.000045
centerloss: 11.005870819091797
Testing acc: 0.7079741379310345
Epoch: [149/150]---src, softmax_loss: 0.000044
centerloss: 11.649832916259765
Training complete in 0m 59s


In [ ]:
#'amazon', 'webcam','dslr','cad'
types='softmax'
source_domain='dslr'
target_domain='amazon'
random_seed=1
weight=0
evaluate(types,source_domain,target_domain,weight,random_seed)


110
928
Types: softmax
Testing acc: 0.8760775862068966
Epoch: [146/150]---src, softmax_loss: 0.000052
centerloss: 15.379467496004972
Testing acc: 0.8760775862068966
Epoch: [147/150]---src, softmax_loss: 0.000051
centerloss: 15.728529774058948
Testing acc: 0.8760775862068966
Epoch: [148/150]---src, softmax_loss: 0.000051
centerloss: 16.27083046653054
Testing acc: 0.8760775862068966
Epoch: [149/150]---src, softmax_loss: 0.000050
centerloss: 16.24046408913352
Training complete in 1m 4s


In [ ]:
#'amazon', 'webcam','dslr','cad'
types='center loss'
source_domain='dslr'
target_domain='amazon'
random_seed=1
weight=0.01
evaluate(types,source_domain,target_domain,weight,random_seed)

110
928
Types: center loss
Testing acc: 0.8717672413793104
Epoch: [146/150]---src, softmax_loss: 0.007156
centerloss: 0.02275822108442133
Testing acc: 0.8674568965517241
Epoch: [147/150]---src, softmax_loss: 0.007148
centerloss: 0.022035855054855346
Testing acc: 0.8706896551724138
Epoch: [148/150]---src, softmax_loss: 0.006973
centerloss: 0.022360619631680574
Testing acc: 0.8706896551724138
Epoch: [149/150]---src, softmax_loss: 0.006926
centerloss: 0.0220579131083055
Training complete in 1m 9s
